In [ ]:
import numpy as np
import math
import time
from joblib import dump, load
import matplotlib.pyplot as plt

In [ ]:
# Determine Parameters
L = 400
N = 400
T = 450
tau = 0.01
h = L / N
lt = int(T / tau)

lp = int(10)

phi = np.zeros((lt + 1, N + 1))

# Initial Condition
d = 100
d_T = 120

phi_0 = np.zeros(N + 1)

for i in range(0, N + 1):
    phi_0[i] = 0.5 * (1 - math.tanh(i - d))
for i in range(d + 1, L + 1):
    phi_0[i] += 0.2 * math.sin((i - d) / d_T * 2 * math.pi)

phi[0] = phi_0

# Compute phi^1
time_start = time.time()

path_operator = "data/Operators/Local/"
P_1 = np.loadtxt(path_operator + "P_1.csv", delimiter=",")
P_2 = np.loadtxt(path_operator + "P_2.csv", delimiter=",")
P_3 = np.loadtxt(path_operator + "P_3.csv", delimiter=",")

for i in range(1, 2):

    phi_0_s = phi_0 ** 2
    phi_1 = np.dot(P_1, phi_0) + np.dot(P_2, phi_0_s) + P_3
    phi[i] = phi_1
    phi_0 = phi_1

# Get Model
A = np.loadtxt("model/Matrix_form/A.csv", delimiter=",")
b = np.loadtxt("model/Matrix_form/b_tilda.csv", delimiter=",")

# %% Compute phi^n

P_1_Prime = np.loadtxt(path_operator + "P_1_Prime.csv", delimiter=",")
P_2_Prime = np.loadtxt(path_operator + "P_2_Prime.csv", delimiter=",")
P_3_Prime = np.loadtxt(path_operator + "P_3_Prime.csv", delimiter=",")
P_4_Prime = np.loadtxt(path_operator + "P_4_Prime.csv", delimiter=",")

period = int(lt / lp)

for i in range(2, lt + 1):

    phi_0_s = phi_0 ** 2
    phi_1_s = phi_1 ** 2

    phi_2 = (
        np.dot(P_1_Prime, phi_0)
        + np.dot(P_2_Prime, phi_0_s)
        + np.dot(P_3_Prime, phi_1)
        + np.dot(P_4_Prime, phi_1_s)
        + np.dot(A, phi_1) + b
    )
    
    phi_0 = phi_1
    phi_1 = phi_2

    if i % period == 0:
        print(i)

    phi[i] = phi_2

time_end = time.time()

print("time = %f" % (time_end - time_start))

In [ ]:
x = np.linspace(0,400,401)

for i in range(0,45001,3000):
    plt.plot(x, phi[i], "r")
plt.xlabel("n")
plt.ylabel("$\phi_{BC}$")
plt.show()

In [ ]:
print(phi.shape)
np.savetxt("data/Solution/Local/phi_BC_init_sin.csv", phi, delimiter=",")

In [ ]:
# Get exact Solution
L = 1000
N = 1000
T = 450
tau = 0.01
h = L / N
lt = int(T / tau)

lp = int(10)

phi = np.zeros((lt + 1, N + 1))


# Initial Conditions
d = int(N / 10)

phi_0 = np.zeros(N + 1)

for i in range(0, N + 1):
    phi_0[i] = 0.5 * (1 - math.tanh(i - d))
for i in range(d + 1, L + 1):
    phi_0[i] += 0.2 * math.sin((i - d) / d_T * 2 * math.pi)

phi[0] = phi_0


# Compute phi^1

path_operator = "data/Operators/Global/"

P_1 = np.loadtxt(path_operator + "P_1.csv", delimiter=",")
P_2 = np.loadtxt(path_operator + "P_2.csv", delimiter=",")
P_3 = np.loadtxt(path_operator + "P_3.csv", delimiter=",")

phi_0_s = phi_0 ** 2

phi_1 = np.dot(P_1, phi_0) + np.dot(P_2, phi_0_s) + P_3
phi[1] = phi_1


# Compute phi^n

time_start = time.time()

P_1_Prime = np.loadtxt(path_operator + "P_1_Prime.csv", delimiter=",")
P_2_Prime = np.loadtxt(path_operator + "P_2_Prime.csv", delimiter=",")
P_3_Prime = np.loadtxt(path_operator + "P_3_Prime.csv", delimiter=",")
P_4_Prime = np.loadtxt(path_operator + "P_4_Prime.csv", delimiter=",")
P_5_Prime = np.loadtxt(path_operator + "P_5_Prime.csv", delimiter=",")

period = int(lt / lp)

for i in range(2, lt + 1):

    phi_0_s = phi_0 ** 2
    phi_1_s = phi_1 ** 2

    phi_2 = (
        np.dot(P_1_Prime, phi_0)
        + np.dot(P_2_Prime, phi_0_s)
        + np.dot(P_3_Prime, phi_1)
        + np.dot(P_4_Prime, phi_1_s)
        + P_5_Prime
    )

    phi_0 = phi_1
    phi_1 = phi_2

    if i % period == 0:
        print(i)

    phi[i] = phi_2

time_end = time.time()

print("time = %f" % (time_end - time_start))

In [ ]:
x = np.linspace(0,1000,1001)
for i in range(0,45001,3000):
    plt.plot(x, phi[i], "r")
plt.xlim(0,401)
plt.xlabel("n")
plt.ylabel("$\phi_{exact}$")
plt.show()

In [ ]:
phi_exact = phi[:,:401]
print(phi_exact.shape)
path_solution = "data/Solution/Local/"
np.savetxt(path_solution + "phi_exact_init_sin.csv", phi_exact, delimiter=",")